In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
from __future__ import annotations

from typing import Dict, List, Sequence
import heapq

from sentence_transformers import SentenceTransformer, util

ModelName = "distiluse-base-multilingual-cased-v1"


def _prepare_text(tags: Sequence[str]) -> str:
    return " ".join(tags)


def build_embeddings(book_db: Dict[str, Sequence[str]],
                     model_name: str = ModelName):
    model = SentenceTransformer(model_name)

    titles, tag_texts = zip(*(
        (title, _prepare_text(tags)) for title, tags in book_db.items()
    ))
    embeds = model.encode(list(tag_texts), convert_to_tensor=True, show_progress_bar=False)
    book_vecs = {
        title: embed for title, embed in zip(titles, embeds)
    }
    return model, book_vecs


def recommend(book_db: Dict[str, Sequence[str]],
              book_vecs: Dict[str, "torch.Tensor"],
              input_titles: Sequence[str],
              top_k: int = 5) -> List[str]:
    if not input_titles:
        raise ValueError("input_titles must contain at least 1 title")

    vectors = [book_vecs[t] for t in input_titles if t in book_vecs]
    if not vectors:
        raise KeyError("None of the provided titles were found in the database")

    import torch
    pref = torch.stack(vectors).mean(dim=0, keepdim=True)

    sims = util.cos_sim(pref, torch.stack(list(book_vecs.values())))[0]

    candidates = [title for title in book_db.keys() if title not in input_titles]
    candidate_sims = [s.item() for t, s in zip(book_db.keys(), sims) if t not in input_titles]

    top = heapq.nlargest(top_k, zip(candidate_sims, candidates))
    return [title for _, title in top]


In [ ]:
book_db_demo = {
        "Гарри Поттер и философский камень – Дж. К. Роулинг": ["Фэнтези", "Подростковое фэнтези"],
        "Перси Джексон и похититель молний – Рик Риордан": ["Фэнтези", "Подростковое фэнтези"],
        "Властелин колец – Дж. Р. Р. Толкин": ["Фэнтези", "Эпическое фэнтези"],
        "Убить пересмешника – Харпер Ли": ["Классика", "Социальная драма"],
        "451° по Фаренгейту – Рэй Брэдбери": ["Научная фантастика", "Антиутопия"],
        "Дюнкерк – Антони Бивор": ["Нон‑фикшн", "Военная история"],
        "Дюна – Фрэнк Герберт": ["Научная фантастика", "Эпическая фантастика"],
}

model, book_vecs = build_embeddings(book_db_demo)

seed_books = [
        "Убить пересмешника – Харпер Ли",
]
print("Input:", " | ".join(seed_books))
print("Recommendations: \n  - " + "\n  - ".join(recommend(book_db_demo, book_vecs, seed_books, top_k=5)))

Input: Убить пересмешника – Харпер Ли
Recommendations: 
  - Дюна – Фрэнк Герберт
  - Властелин колец – Дж. Р. Р. Толкин
  - Перси Джексон и похититель молний – Рик Риордан
  - Гарри Поттер и философский камень – Дж. К. Роулинг
  - 451° по Фаренгейту – Рэй Брэдбери


In [5]:
book_db = {'«Древний Египет в Эрмитаже. Новые открытия», Андрей Большаков': ['Востоковедение', 'История Древнего Востока'],
 '«Пирамиды над раскопом. Египет глазами российских археологов», Сергей Ветохов, Максим Лебедев, Светлана Малых': ['Востоковедение',
                                                                                                                   'История '
                                                                                                                   'Древнего '
                                                                                                                   'Востока'],
 '«Месопотамия и Иран в VII–IV вв. до н.э.: Социальные институты и идеология», Мухаммед Дандамаев': ['Востоковедение',
                                                                                                     'История Древнего '
                                                                                                     'Востока'],
 '«Очерки по истории Южной Месопотамии эпохи ранней древности (VII тыс. до н.э. – середина II тыс. до н.э.)», Нэлли Козырева': ['Востоковедение',
                                                                                                                                'История '
                                                                                                                                'Древнего '
                                                                                                                                'Востока'],
 '«Языки мира: Древние реликтовые языки Передней Азии», ред. кол.: Николай Казанский, Андрей Кибрик, Юрий Коряков': ['Востоковедение',
                                                                                                                     'История '
                                                                                                                     'Древнего '
                                                                                                                     'Востока'],
 '«Японская литература VIII-XVI вв.: Начало и развитие традиций», Владислав Горегляд': ['Востоковедение',
                                                                                        'Японоведение'],
 '«Красотой Японии рожденный», Татьяна Григорьева': ['Востоковедение', 'Японоведение'],
 '«Миллионы, миллионы японцев», Жан-Пьер Шаброль': ['Востоковедение', 'Японоведение'],
 '«Японцы открывают Европу», Дональд Кин': ['Востоковедение', 'Японоведение'],
 '«Мир блистательного принца: придворная жизнь в древней Японии», Айван Морис': ['Востоковедение', 'Японоведение'],
 '«Россия — Иран (Персия): очерки истории отношений (XVI — начало XXI вв.)», Игорь Базиленко': ['Востоковедение',
                                                                                                'Ближний Восток. '
                                                                                                'История, философия, '
                                                                                                'историография.'],
 '«Магриб-Машрик-Россия. Материалы по историографии и истории арабского мира», Николай Дьяков': ['Востоковедение',
                                                                                                 'Ближний Восток. '
                                                                                                 'История, философия, '
                                                                                                 'историография.'],
 '«Образ Европы и России в произведениях османских авторов XVIII в.», Константин Жуков, Александра Жевелева': ['Востоковедение',
                                                                                                               'Ближний '
                                                                                                               'Восток. '
                                                                                                               'История, '
                                                                                                               'философия, '
                                                                                                               'историография.'],
 '«Философия идеальной власти. Политические концепции средневекового ислама», Тигран Туманян': ['Востоковедение',
                                                                                                'Ближний Восток. '
                                                                                                'История, философия, '
                                                                                                'историография.'],
 '«Новая Россия и мусульманский Ближний Восток (арабские страны, Турция, Иран) на рубеже XX-XXI вв.», Игорь Базиленко, Николай Дьяков, Константин Жуков': ['Востоковедение',
                                                                                                                                                           'Ближний '
                                                                                                                                                           'Восток. '
                                                                                                                                                           'История, '
                                                                                                                                                           'философия, '
                                                                                                                                                           'историография.'],
 '« »': ['nan', 'nan'],
 '«Сиам по картам. (“Siam Mapped: A History of the Geo-Body of a Nation”)», Тонгчай Виничакун': ['Востоковедение',
                                                                                                 'Юго-Восточная Азия: '
                                                                                                 'история, религия, '
                                                                                                 'философия, '
                                                                                                 'культрология'],
 "«Вессантара-джатака и концепция совершенного человека. (“Thailand's Theory of Monarchy: The Vessantara Jātaka and the Idea of the Perfect Man”)», Патрик Джори": ['Востоковедение',
                                                                                                                                                                    'Юго-Восточная '
                                                                                                                                                                    'Азия: '
                                                                                                                                                                    'история, '
                                                                                                                                                                    'религия, '
                                                                                                                                                                    'философия, '
                                                                                                                                                                    'культрология'],
 '«Четыре правления», Кыкрит Прамот': ['Востоковедение',
                                       'Юго-Восточная Азия: история, религия, философия, культрология'],
 '«Очерки культуры тайцев Таиланда», Елена Иванова': ['Востоковедение',
                                                      'Юго-Восточная Азия: история, религия, философия, культрология'],
 '«Тайский буддизм», Владимир Корнев': ['Востоковедение',
                                        'Юго-Восточная Азия: история, религия, философия, культрология'],
 '«История корейской традиционной литературы (до XX в.)», Аделаида Троцевич': ['Востоковедение', 'Корееведение'],
 '«С блокнотом по Корее», Сергей Курбанов': ['Востоковедение', 'Корееведение'],
 '«Миндаль», Сон Вон Пхён': ['Востоковедение', 'Корееведение'],
 '«Человеческие поступки», Хан Ган': ['Востоковедение', 'Корееведение'],
 '«Библиотека музыкальных инструментов», Ким Чжун Хёк': ['Востоковедение', 'Корееведение'],
 '«Китайцы. Особенности национальной психологии», Николай Спешнев': ['Востоковедение',
                                                                     'Культура и традиции современного Китая'],
 '«Словарь китайского гурмана», Цуй Дайюань': ['Востоковедение', 'Культура и традиции современного Китая'],
 '«Десять слов про Китай», Юй Хуа': ['Востоковедение', 'Культура и традиции современного Китая'],
 '«Большой мир китайского языка», Ольга Завьялова': ['Востоковедение', 'История иероглифики и китайского языка'],
 '«История культуры Китая», Марина Кравцова': ['Востоковедение', 'История и культура Китая'],
 '«Древнетюркские рунические памятники как источник по истории Средней Азии», Сергей Кляшторный': ['Востоковедение',
                                                                                                   'История '
                                                                                                   'возникновения и '
                                                                                                   'развития письма у '
                                                                                                   'тюркских народов: '
                                                                                                   'основные системы'],
 '«Исследования по истории тюркских языков XI–XIV вв.», Эмир Наджип': ['Востоковедение',
                                                                       'История возникновения и развития письма у '
                                                                       'тюркских народов: основные системы'],
 '«История письменности. От рисуночного письма к полноценному алфавиту», Игнас Джей Гельб': ['Востоковедение',
                                                                                             'История возникновения и '
                                                                                             'развития письма у '
                                                                                             'тюркских народов: '
                                                                                             'основные системы'],
 '«Рунические письменности евразийских степей», Игорь Кызласов': ['Востоковедение',
                                                                  'История возникновения и развития письма у тюркских '
                                                                  'народов: основные системы'],
 '«Государства и народы евразийских степей: от древности к Новому времени», Сергей Кляшторный, Турсун Султанов': ['Востоковедение',
                                                                                                                  'История '
                                                                                                                  'возникновения '
                                                                                                                  'и '
                                                                                                                  'развития '
                                                                                                                  'письма '
                                                                                                                  'у '
                                                                                                                  'тюркских '
                                                                                                                  'народов: '
                                                                                                                  'основные '
                                                                                                                  'системы'],
 '«Перевод контекстуально-обусловленных английских слов», Борис Погодин': ['Филология',
                                                                           'Практика перевода в паре английский '
                                                                           'язык-русский язык'],
 '«Искусственный интеллект: современный подход (AIMA)», Стюарт Рассел, Питер Норвиг': ['Филология',
                                                                                       'Искусственный интеллект'],
 '«Лингвистические задачи», Андрей Зализняк': ['Филология',
                                               'Полезная научно-популярная литература (для лингвистов и переводчиков)'],
 '«Лингвистические детективы», Николай Шанский': ['Филология',
                                                  'Полезная научно-популярная литература (для лингвистов и '
                                                  'переводчиков)'],
 '«Искусство создания языков: от вымершего языка высших классов до наречия кровожадных воинов-кочевников», Дэвид Питерсон': ['Филология',
                                                                                                                             'Полезная '
                                                                                                                             'научно-популярная '
                                                                                                                             'литература '
                                                                                                                             '(для '
                                                                                                                             'лингвистов '
                                                                                                                             'и '
                                                                                                                             'переводчиков)'],
 '«Русская литература Золотого века. Лекции», Владимир Маркович': ['Филология',
                                                                   'История русской литературы первой половины XIX '
                                                                   'века'],
 '«Литературная герменевтика: теория и практика», Екатерина Ляпушкина': ['Филология', 'Литературная герменевтика'],
 '«Речь и молчание: Сюжеты и мифы русской словесности», Мария Виролайнен': ['Филология',
                                                                            'Литературоведение, история и культура '
                                                                            'русской литературы'],
 '«На пороге как бы двойного бытия…». О творчестве И. А. Гончарова и его современников», Михаил Отрадин': ['Филология',
                                                                                                           'История '
                                                                                                           'русской '
                                                                                                           'литературы'],
 '«Русский святочный рассказ. Становление жанра», Елена Душечкина': ['Филология', 'История литературных жанров'],
 '«Магия тишины. Путешествие Каспара Давида Фридриха сквозь время», Флориан Иллиес': ['Филология',
                                                                                      'История зарубежной литературы'],
 '«Каспар Давид Фридрих. Странник немецкого романтизма», Александра Коншакова': ['Филология',
                                                                                 'Искусствоведение и '
                                                                                 'литературоведение'],
 '«Музыка: диалог с Богом. От архаики до электроники», Роман Насонов': ['Филология', 'История музыки'],
 '«Немецкий романтизм и современная мистика», Виктор Жирмунский': ['Филология',
                                                                   'История зарубежной литературы и философии'],
 '«Волшебное сказание о силе Музыки», Вильгельм Вакенродер': ['Филология', 'История музыки'],
 '«Педро Парамо», Хуан Рульфо': ['Филология', 'Зарубежная литература Латиноамериканская литература'],
 '«Пятеро, которые молчали», Мигель Отеро Сильва': ['Филология', 'Зарубежная литература Латиноамериканская литература'],
 '«Концерт барокко», Алехо Карпентьер': ['Филология', 'Зарубежная литература Латиноамериканская литература'],
 '«Преследователь», Хулио Кортасар': ['Филология', 'Зарубежная литература Латиноамериканская литература'],
 '«Зеленый дом», Варгас Льоса': ['Филология', 'Зарубежная литература Латиноамериканская литература'],
 '«Русская ментальность в языке и тексте», Владимир Колесов': ['Филология', 'Русский язык'],
 '«Русская диалектология: говорит бабушка Марфа, а мы комментируем», Валентин Трубинский': ['Филология',
                                                                                            'Русский язык'],
 '«Дар мнемозины. Романы Набокова в контексте русской автобиографической традиции», Борис Аверин': ['Филология',
                                                                                                    'Русский язык'],
 '«Давайте говорить правильно! Трудности современного русского произношения и ударения», Людмила Вербицкая, Наталья Богданова-Бегларян, Галина Скляревская': ['Филология',
                                                                                                                                                              'Русский '
                                                                                                                                                              'язык'],
 '«Лавр», Евгений Водолазкин': ['Филология', 'Русский язык'],
 '«Русская речь в средствах массовой информации: Стилистический аспект», под ред. Владимира Конькова': ['Журналистика',
                                                                                                        'Медиалингвистика'],
 '«Русская речь в средствах массовой информации. Речевые системы и структуры», под ред. Владимира Конькова, Анны Потсар': ['Журналистика',
                                                                                                                           'Медиалингвистика'],
 '«Стилистика и литературное редактирование в 2 томах», Учебник для академического бакалавриата под ред. Лилии Дускаевой': ['Журналистика',
                                                                                                                            'Медиалингвистика'],
 '«Медиалингвистика в терминах и понятиях», под ред. Лилии Дускаевой': ['Журналистика', 'Медиалингвистика'],
 '«Вся наша жизнь — в мемах: Лингвистический анализ весёлых картинок», Наталья Прокофьева, Екатерина Щеглова': ['Журналистика',
                                                                                                                'Медиалингвистика'],
 '«Когда умрут газеты», Андрей Мирошниченко': ['Журналистика', 'Теория и практика современой журналистики'],
 '«Коммуникации на раз-два-три», Игорь Писарский': ['Журналистика', 'Теория и практика современой журналистики'],
 '«Добыча. Всемирная история борьбы за нефть, деньги, власть», Дэниел Ергин': ['Журналистика',
                                                                               'Теория и практика современой '
                                                                               'журналистики'],
 '«TED Talks. Слова меняют мир», Крис Андерсон': ['Журналистика', 'Теория и практика современой журналистики'],
 '«Chat GPT на каждый день», Дамир Халилов': ['Журналистика', 'Теория и практика современой журналистики'],
 '«Машинное обучение без лишних слов», Андрей Бурков': ['Бизнес-аналитика и большие данные',
                                                        'Бизнес-аналитика и большие данные'],
 '«Доверительное А/В-тестирование. Практическое руководство по контролируемым экспериментам», Рон Кохави, Диана Тан, Я Сюй': ['Бизнес-аналитика '
                                                                                                                              'и '
                                                                                                                              'большие '
                                                                                                                              'данные',
                                                                                                                              'Маркетинг '
                                                                                                                              'Машинное '
                                                                                                                              'обучение '
                                                                                                                              'Анализ '
                                                                                                                              'данных'],
 '«DAMA-DMBOK: Свод знаний по управлению данными», справочник DAMA International': ['Бизнес-аналитика и большие данные',
                                                                                    'Бизнес-аналитика и большие '
                                                                                    'данные'],
 '«Статистика для менеджеров с использованием MICROSOFT Excel», Дэвид Левин, Дэвид Штефан': ['Бизнес-аналитика и '
                                                                                             'большие данные',
                                                                                             'Маркетинг Машинное '
                                                                                             'обучение Анализ данных'],
 '«Наука о данных и аналитика больших данных: обнаружение, анализ, визуализация и представление данных. Data Science and Big Data Analytics: Discovering, Analyzing, Visualizing and Presenting Data», EMC Services Education': ['Бизнес-аналитика '
                                                                                                                                                                                                                                 'и '
                                                                                                                                                                                                                                 'большие '
                                                                                                                                                                                                                                 'данные',
                                                                                                                                                                                                                                 'Маркетинг '
                                                                                                                                                                                                                                 'Машинное '
                                                                                                                                                                                                                                 'обучение '
                                                                                                                                                                                                                                 'Анализ '
                                                                                                                                                                                                                                 'данных'],
 '«Практика управления человеческими ресурсами», Майкл Армстронг': ['Менеджмент', 'Управление людьми'],
 '«Организационное поведение: учебник для вузов», Габдельахат Латфуллин, Ольга Громова': ['Менеджмент',
                                                                                          'Управление людьми'],
 '«Управление человеческими ресурсами: учебник», Владимир Веснин': ['Менеджмент', 'Управление людьми'],
 '«HR как он есть», Татьяна Кожевникова': ['Менеджмент', 'Управление людьми'],
 '«HR-аналитика: Практическое руководство по работе с персоналом на основе больших данных», Надим Кхан, Дэйв Миллнер': ['Менеджмент',
                                                                                                                        'Управление '
                                                                                                                        'людьми'],
 '«Москва — Петроград. Два центра отечественной реставрации», Виктор Рославский': ['Музеология и охрана объектов '
                                                                                   'культурного и природного наследия',
                                                                                   'Охрана и реставрация памятников '
                                                                                   'архитектуры Петербурга'],
 '«Памятники. Люди. События: из записок музейного работника», Марина Тихомирова': ['Музеология и охрана объектов '
                                                                                   'культурного и природного наследия',
                                                                                   'Охрана и реставрация памятников '
                                                                                   'архитектуры Петербурга'],
 '«Консервация и реставрация музейных коллекций», Мстислав Фармаковский': ['Музеология и охрана объектов культурного и '
                                                                           'природного наследия',
                                                                           'Климатические параметры и биологический '
                                                                           'контроль в музеях'],
 '«Методы биологического контроля и анализа в музейной работе», Ирина Кирцидели, Надежда Соловьёва, Ирина Дымская': ['Музеология '
                                                                                                                     'и '
                                                                                                                     'охрана '
                                                                                                                     'объектов '
                                                                                                                     'культурного '
                                                                                                                     'и '
                                                                                                                     'природного '
                                                                                                                     'наследия',
                                                                                                                     'Климатические '
                                                                                                                     'параметры '
                                                                                                                     'и '
                                                                                                                     'биологический '
                                                                                                                     'контроль '
                                                                                                                     'в '
                                                                                                                     'музеях'],
 '«Система комплексного профилактического обслуживания памятников деревянного зодчества», Александр Любимцев, Маргарита Кистерная': ['Музеология '
                                                                                                                                     'и '
                                                                                                                                     'охрана '
                                                                                                                                     'объектов '
                                                                                                                                     'культурного '
                                                                                                                                     'и '
                                                                                                                                     'природного '
                                                                                                                                     'наследия',
                                                                                                                                     'Климатические '
                                                                                                                                     'параметры '
                                                                                                                                     'и '
                                                                                                                                     'биологический '
                                                                                                                                     'контроль '
                                                                                                                                     'в '
                                                                                                                                     'музеях'],
 '«И.В. Цветаев создаёт музей», Александра Демская, Людмила Смирнова': ['Музеология и охрана объектов культурного и '
                                                                          'природного наследия',
                                                                          'Музееведение. История создания и развития '
                                                                          'музеев России'],
 '«История Картинной галереи Эрмитажа», Владимир Левинсон-Лессинг': ['Музеология и охрана объектов культурного и '
                                                                     'природного наследия',
                                                                     'Музееведение. История создания и развития музеев '
                                                                     'России'],
 '«Музей в мировой культуре», Тамара Юренева': ['Музеология и охрана объектов культурного и природного наследия',
                                                'Музееведение. История создания и развития музеев России'],
 '«Музей как объект культуры. Искусство экспозиционного ансамбля — начало XXI века», Мария Майстровская': ['Музеология '
                                                                                                           'и охрана '
                                                                                                           'объектов '
                                                                                                           'культурного '
                                                                                                           'и '
                                                                                                           'природного '
                                                                                                           'наследия',
                                                                                                           'Музееведение. '
                                                                                                           'Теория и '
                                                                                                           'практика '
                                                                                                           'экспозиционной '
                                                                                                           'деятельности.'],
 '«Три музейных десятилетия. Архитектура и экспозиция музеев современного искусства 1970-х–1990-х гг.», Мария Микаелян': ['Музеология '
                                                                                                                          'и '
                                                                                                                          'охрана '
                                                                                                                          'объектов '
                                                                                                                          'культурного '
                                                                                                                          'и '
                                                                                                                          'природного '
                                                                                                                          'наследия',
                                                                                                                          'Музееведение. '
                                                                                                                          'Теория '
                                                                                                                          'и '
                                                                                                                          'практика '
                                                                                                                          'экспозиционной '
                                                                                                                          'деятельности.'],
 '«Философия русской революции», ред.: Игорь Осипов, Сергей Погодин': ['Философия', 'История конфликтологии'],
 '«Конфликты в условиях трансформации современного российского общества», ред.: Андрей Алейников, Александр Стребков': ['Философия',
                                                                                                                        'История '
                                                                                                                        'конфликтологии'],
 '«Аксиология российского конституционализма. История и современность.», ред.: Сергей Дудник, Игорь Осипов': ['Философия',
                                                                                                              'История '
                                                                                                              'конфликтологии'],
 '«Политическая иммунология», Борис Марков': ['Философия', 'Философия политики'],
 '«Цивилизация права и развитие России», Валерий Зорькин': ['Философия', 'Философия политики'],
 '«Вода в ледниках. Методы и результаты геофизических и дистанционных исследований», Андрей Глазовский, Юрий Мачерет': ['Гидрология '
                                                                                                                        'суши, '
                                                                                                                        'ледники, '
                                                                                                                        'гляциология',
                                                                                                                        'Физика '
                                                                                                                        'ледников'],
 '«Радиозондирование ледников», Юрий Мачерет': ['Гидрология суши, ледники, гляциология', 'Физика ледников'],
 '«Антарктическое подледниковое озеро Восток. Гляциология, биология, планетология. История открытия и исследования озера Восток», Игорь Зотиков': ['Гидрология '
                                                                                                                                                   'суши, '
                                                                                                                                                   'ледники, '
                                                                                                                                                   'гляциология',
                                                                                                                                                   'Физика '
                                                                                                                                                   'ледников'],
 '«Динамика масс льда», Уильям Бадд': ['Гидрология суши, ледники, гляциология', 'Физика ледников'],
 '«Физика ледников», Уильям Патерсон': ['Гидрология суши, ледники, гляциология', 'Физика ледников'],
 '«Территория», Олег Куваев': ['Минералогия, кристаллография', 'О минералах и кристаллах'],
 '«Занимательная минералогия», Александр Ферсман': ['Минералогия, кристаллография', 'О минералах и кристаллах'],
 '«История Земли. От звездной пыли — к живой планете. Первые 4 500 000 000 лет», Роберт Хейзен': ['Минералогия, '
                                                                                                  'кристаллография',
                                                                                                  'О минералах и '
                                                                                                  'кристаллах'],
 '«Графика», Морис Корнелиус Эшер': ['Минералогия, кристаллография', 'О минералах и кристаллах'],
 '«Общая кристаллохимия»,Станислав Филатов, Сергей Кривовичев, Римма Бубнова': ['Минералогия, кристаллография',
                                                                                'О минералах и кристаллах'],
 '«Девелопмент недвижимости и комплексное развитие территорий», Сергей Максимов': ['Землеустройство и кадастры',
                                                                                   'Всё о недвижимости'],
 '«Экономика недвижимости», Сергей Максимов': ['Землеустройство и кадастры', 'Всё о недвижимости'],
 '«Управление недвижимостью», Сергей Максимов': ['Землеустройство и кадастры', 'Всё о недвижимости'],
 '«Земельная политика», Станислав Липски': ['Землеустройство и кадастры', 'Всё о недвижимости'],
 '«Основы кадастра недвижимости», Ольга Миклашевская, Александр Сизов': ['Землеустройство и кадастры',
                                                                         'Всё о недвижимости'],
 '«Растворы, минералы, равновесия», Роберт Гаррелс, Чарльз Крайст': ['Геохимия',
                                                                     'Термодинамика геохимических процессов'],
 '«Основы физической геохимии», Вилен Жариков': ['Геохимия', 'Термодинамика геохимических процессов'],
 '«Моделирование геохимических и биогеохимических реакций. Geochemical and biogeochemical Reaction Modeling», Крэйг Бетке': ['Геохимия',
                                                                                                                             'Термодинамика '
                                                                                                                             'геохимических '
                                                                                                                             'процессов'],
 '«Термодинамическое моделирование в геохимии: теория, алгоритмы, программное обеспечение, приложения», Константин Чудненко': ['Геохимия',
                                                                                                                               'Термодинамика '
                                                                                                                               'геохимических '
                                                                                                                               'процессов'],
 '«Трактат по геохимии. Treatise on Geochemistry», Хенрих Холленд, Карл Турекян': ['Геохимия',
                                                                                   'Термодинамика геохимических '
                                                                                   'процессов'],
 '«Похвала «глупости» хромосомы. Исповедь непокорной молекулы», Антонио Лима-де-Фариа': ['Цитология и гистология',
                                                                                         'Биология хромосом и '
                                                                                         'генетики'],
 '«Хромосомы. Структура и функции» Дмитрий Коряков, Игорь Жимулев': ['Цитология и гистология',
                                                                     'Биология хромосом и генетики'],
 '«После человека остаётся только слово: воспоминания, дневниковые записи, путевые заметки, рассказы», Евгений Ананьев': ['Цитология '
                                                                                                                          'и '
                                                                                                                          'гистология',
                                                                                                                          'Биология '
                                                                                                                          'хромосом '
                                                                                                                          'и '
                                                                                                                          'генетики'],
 '«Ретроспектива генетики», Сергей Инге-Вечтомов': ['Цитология и гистология', 'Биология хромосом и генетики'],
 '«Наукообразная чушь. Разоблачение мошенничества, предвзятости, недобросовестности и хайпа в науке», Стюарт Ричи': ['Цитология '
                                                                                                                     'и '
                                                                                                                     'гистология',
                                                                                                                     'Биология '
                                                                                                                     'хромосом '
                                                                                                                     'и '
                                                                                                                     'генетики'],
 '«Лекции по гидроаэромеханике», Сергей Валландер': ['Математика', 'Неравновесная физико-химическая аэромеханика'],
 '«Математическая теория процессов переноса в газах», Дж. Ферцигер, Г. Капер': ['Математика',
                                                                                'Неравновесная физико-химическая '
                                                                                'аэромеханика'],
 '«Релаксационные процессы в ударных волнах», Евгений Ступоченко, Сталий Лосев, Алексей Осипов': ['Математика',
                                                                                                  'Неравновесная '
                                                                                                  'физико-химическая '
                                                                                                  'аэромеханика'],
 '«Кинетическая теория процессов переноса и релаксации в потоках неравновесных реагирующих газов», Екатерина Нагнибеда, Елена Кустова': ['Математика',
                                                                                                                                         'Неравновесная '
                                                                                                                                         'физико-химическая '
                                                                                                                                         'аэромеханика'],
 '«Основные математические преобразования в кинетической теории газов», Елена Кустова, Мария Мехоношина': ['Математика',
                                                                                                           'Неравновесная '
                                                                                                           'физико-химическая '
                                                                                                           'аэромеханика'],
 '«Азы физики», Лев Окунь': ['Физика', 'Введение в современную теоретическую физику'],
 '«Объяснение Вселенной: от Фалеса до Ньютона», Дмитрий Климушкин': ['Физика', 'История физики (общие вопросы)'],
 '«Под знаком кванта», Леонид Пономарёв': ['Физика', 'История физики (отдельные разделы)'],
 '«Интерстеллар: наука за кадром», Кип Торн': ['Физика', 'Черные дыры'],
 '«Физика черных дыр», Игорь Новиков, Валерий Фролов': ['Физика', 'Черные дыры'],
 '«Отличная квантовая механика», Александр Львовский': ['Физика',
                                                        'Квантовая физика, квантовая механика и квантовая оптика'],
 '«Современная теоретическая химия в современном изложении», Денис Тихонов': ['Физика', 'Теоретичеая физическая химия'],
 '«Классическая и современная отптика. Classical and Modern Optics», Дэниель Стек': ['Физика', 'Классическая оптика'],
 '«Квантовые вычисления и квантовая информация», Майкл Нильсен, Иссак Чанг': ['Физика',
                                                                              'Квантовая информатика и квантовые '
                                                                              'вычисления'],
 '«Задачи и их решения в квантовых вычислениях и квантовой теории информации», Вилли-Ханс Стиб, Йорик Харди': ['Физика',
                                                                                                               'Квантовая '
                                                                                                               'теория '
                                                                                                               'информации'],
 '«Занимательная ядерная физика», Константин Мухин': ['Физика', 'Научно-популярная литература по физике и не только'],
 '«От мечты к открытию: Как стать учёным (Hans Selye, From Dream to Discovery: On Being a Scientist)», Ганс Селье': ['Физика',
                                                                                                                     'Научно-популярная '
                                                                                                                     'литература '
                                                                                                                     'по '
                                                                                                                     'физике '
                                                                                                                     'и '
                                                                                                                     'не '
                                                                                                                     'только'],
 '«КЭД – странная теория света и вещества», Ричард Фейнман': ['Физика',
                                                              'Научно-популярная литература по физике и не только'],
 '«Космические лучи», Бруно Росси': ['Физика', 'Научно-популярная литература по физике и не только'],
 '«Миф о Сизифе из книги Бунтующий человек», Альбер Камю': ['Физика',
                                                            'Научно-популярная литература по физике и не только'],
 '«Чем пахнет дождь?», Саймон Кинг, Клэр Насир': ['Физика, физика атмосферы', 'Озон, климат и космическая погода'],
 '«Химическая физика озонового слоя», Игорь Ларин': ['Физика, физика атмосферы', 'Озон, климат и космическая погода'],
 '«Космическая погода и наша жизнь», Борис Владимирский, Наталья Темурьянц, Виктор Мартынюк': ['Физика, физика '
                                                                                               'атмосферы',
                                                                                               'Озон, климат и '
                                                                                               'космическая погода'],
 '«Неудобная правда. Кризис глобального потепления», Альберт Гор': ['Физика, физика атмосферы',
                                                                    'Озон, климат и космическая погода'],
 '«Жизнь на нашей планете. Моё предупреждение миру на грани катастрофы», Дэвид Аттенборо': ['Физика, физика атмосферы',
                                                                                            'Озон, климат и '
                                                                                            'космическая погода'],
 '«Случайные блуждания: непредсказуемый путь к графену», Андрей Гейм': ['Физика',
                                                                        'Современная физика твёрдого тела: от '
                                                                        'вдохновения к новым открытиям'],
 '«"Из чего это сделано?" Удивительные материалы, из которых построена современная цивилизация», Марк Медовник': ['Физика',
                                                                                                                  'Современная '
                                                                                                                  'физика '
                                                                                                                  'твёрдого '
                                                                                                                  'тела: '
                                                                                                                  'от '
                                                                                                                  'вдохновения '
                                                                                                                  'к '
                                                                                                                  'новым '
                                                                                                                  'открытиям'],
 '«Барьеры. От кристалла до интегральной схемы», Михаил Левинштейн, Григорий Симин': ['Физика',
                                                                                      'Современная физика твёрдого '
                                                                                      'тела: от вдохновения к новым '
                                                                                      'открытиям'],
 '«Спинтроника: учебное пособие», Виктор Борисенко, Александр Данилюк, Дмитрий Мигас': ['Физика',
                                                                                        'Современная физика твёрдого '
                                                                                        'тела: от вдохновения к новым '
                                                                                        'открытиям'],
 '«Современный справочник по физике конденсированного состояния. Modern Condensed Matter Physics», Стивен Гирвин, Кун Янг': ['Физика',
                                                                                                                             'Современная '
                                                                                                                             'физика '
                                                                                                                             'твёрдого '
                                                                                                                             'тела: '
                                                                                                                             'от '
                                                                                                                             'вдохновения '
                                                                                                                             'к '
                                                                                                                             'новым '
                                                                                                                             'открытиям'],
 '«Сарматы. Древний народ юга России. The Sarmatians», Тадеуш Сулимирский': ['История', 'Сарматская археология'],
 '«О времени первого активного выступления сарматов в Поднепровье по свидетельствам античных письменных источников», Дмитрий Мачинский': ['История',
                                                                                                                                          'Сарматская '
                                                                                                                                          'археология'],
 '«Сарматская концепция М. И. Ростовцева и прохоровские курганы. Парфянский выстрел», Вадим Зуев': ['История',
                                                                                                    'Сарматская '
                                                                                                    'археология'],
 '«Сарматы», Анатолий Скрипкин': ['История', 'Сарматская археология'],
 '«Сарматы. Первая тяжёлая конница степей», Александр Нефёдкин': ['История', 'Сарматская археология'],
 '«Финно-угорские племена в составе Древней Руси: К истории славяно-финских этнокультурных связей. Историко-археологические очерки», Евгений Рябинин': ['История',
                                                                                                                                                        'История '
                                                                                                                                                        'и '
                                                                                                                                                        'этнография '
                                                                                                                                                        'Древней '
                                                                                                                                                        'Руси'],
 '«Эпоха викингов в Северной Европе и на Руси», Глеб Лебедев': ['История', 'История и этнография Северной Европы'],
 '«О сагах, скальдах и конунгах: Избранные статьи», Татьяна Джаксон': ['История',
                                                                       'История и этнография стран Скандинавии'],
 '«Монеты. Клады. Коллекции: Очерки нумизматики», Всеволод Потин': ['История', 'История нумизматики'],
 '«Основания регионалистики: Формирование и эволюция историко-культурных зон Европейской России», Василий Булкин, Александр Герд, Глеб Лебедев, Валерий Седых': ['История',
                                                                                                                                                                 'Историко-культурные '
                                                                                                                                                                 'вопросы '
                                                                                                                                                                 'зон '
                                                                                                                                                                 'Европейской '
                                                                                                                                                                 'России'],
 '«Финляндия во Второй мировой войне», Николай Барышников, Владимир Барышников, Владимир Фёдоров': ['История',
                                                                                                    'Финляндия во '
                                                                                                    'Второй мировой '
                                                                                                    'войне'],
 '«От прохладного мира к зимней войне: Восточная политика Финляндии в 1930-е годы», Владимир Барышников': ['История',
                                                                                                           'Финляндия '
                                                                                                           'во Второй '
                                                                                                           'мировой '
                                                                                                           'войне'],
 '«Вступление Финляндии во Вторую мировую войну 1940-1941 гг.», Владимир Барышников': ['История',
                                                                                       'Финляндия во Второй мировой '
                                                                                       'войне'],
 '«Маннергейм и Советский Союз», Владимир Барышников': ['История', 'Финляндия во Второй мировой войне'],
 '«Финская война», Владимир Барышников': ['История', 'Финляндия во Второй мировой войне'],
 '«Кризис и трагедия континента. Тридцатилетняя война (1618-1648) в событиях и коллективной памяти Европы», Юрий Ивонин и Людмила Ивонина': ['История',
                                                                                                                                             'История '
                                                                                                                                             'Тридцатилетней '
                                                                                                                                             'войны'],
 '«Тридцатилетняя война», Андрей Прокопьев': ['История', 'История Тридцатилетней войны'],
 '«Путь к катастрофе Тридцатилетней войны. Кризисное десятилетие 1608-1618 гг.э. Der Weg in die Katastrophe des Dreißigjährigen Krieges. Die Krisendekade 1608-1618», Хайнц Духхард': ['История',
                                                                                                                                                                                       'История '
                                                                                                                                                                                       'Тридцатилетней '
                                                                                                                                                                                       'войны'],
 '«Войны и кризисы. Германия 1600-1715. Kriege und Kriese. Deutschland 1600-1715», Фолькер Пресс': ['История',
                                                                                                    'История '
                                                                                                    'Тридцатилетней '
                                                                                                    'войны'],
 "«Дридцатилетняя война. The Thirty Years' War», Джеффри Паркер": ['История', 'История Тридцатилетней войны'],
 '«Дамы на обочине: Три женских портрета XVII века», Натали Дэвис': ['История',
                                                                     'Ранее Новое время, идеи эпохи Просвещения, '
                                                                     'интеллектуальная история, трансфер идей'],
 '«Поэзия и полиция: Сеть коммуникаций в Париже XVIII века», Роберт Дарнтон': ['История',
                                                                               'Ранее Новое время, идеи эпохи '
                                                                               'Просвещения, интеллектуальная история, '
                                                                               'трансфер идей'],
 '«Литературный тур де Франс. Мир книг накануне Французской революции», Роберт Дарнтон': ['История',
                                                                                          'Ранее Новое время, идеи '
                                                                                          'эпохи Просвещения, '
                                                                                          'интеллектуальная история, '
                                                                                          'трансфер идей'],
 '«Культурные истоки Французской революции», Роже Шартье': ['История',
                                                            'Ранее Новое время, идеи эпохи Просвещения, '
                                                            'интеллектуальная история, трансфер идей'],
 '«Постколониальное Просвещение: колониализм восемнадцатого века и постколониальная теория. The Postcolonial Enlightenment: Eighteenth-century Colonialism and Postcolonial Theory», под редакцией Дэниел Кэри и Линн Феста': ['История',
                                                                                                                                                                                                                               'Ранее '
                                                                                                                                                                                                                               'Новое '
                                                                                                                                                                                                                               'время, '
                                                                                                                                                                                                                               'идеи '
                                                                                                                                                                                                                               'эпохи '
                                                                                                                                                                                                                               'Просвещения, '
                                                                                                                                                                                                                               'интеллектуальная '
                                                                                                                                                                                                                               'история, '
                                                                                                                                                                                                                               'трансфер '
                                                                                                                                                                                                                               'идей'],
 '«Общественное движение при Александре II (1855–1881): исторические очерки», Александр Корнилов': ['История',
                                                                                                    'Общественная '
                                                                                                    'борьба в России '
                                                                                                    'XIX – начала XX '
                                                                                                    'вв.'],
 '«Жандарм литературной республики. Н.К. Михайловский. Жизнь, литература, политическая борьба», Владимир Блохин': ['История',
                                                                                                                   'Общественная '
                                                                                                                   'борьба '
                                                                                                                   'в '
                                                                                                                   'России '
                                                                                                                   'XIX '
                                                                                                                   '– '
                                                                                                                   'начала '
                                                                                                                   'XX '
                                                                                                                   'вв.'],
 '«Российское революционное движение во второй половине XIX века», Владислав Гросул': ['История',
                                                                                       'Общественная борьба в России '
                                                                                       'XIX – начала XX вв.'],
 '«История русской мысли от Просвещения до марксизма», Анджей Валицкий': ['История',
                                                                          'Общественная борьба в России XIX – начала '
                                                                          'XX вв.'],
 '«Романтики, реформаторы, реакционеры. Русская консервативная мысль и политика в царствование Александра I», Александр Мартин': ['История',
                                                                                                                                  'История '
                                                                                                                                  'русского '
                                                                                                                                  'консерватизма '
                                                                                                                                  'XIX '
                                                                                                                                  'в.'],
 '«Другое кино: статьи по истории отечественного кино первой трети XX века», Рашит Янгиров': ['История',
                                                                                              'История и теория кино'],
 '«Труппа Её Величества», Лилия Шитенбург': ['История', 'История и теория кино'],
 '«Психология кинодискурса: факторы выбора, восприятие, воздействие», Тина Кубрак, Владислав Латынов': ['История',
                                                                                                        'История и '
                                                                                                        'теория кино'],
 '«Fine cuts. Интервью о практике европейского киномонтажа», Роджер Криттенден': ['История', 'История и теория кино'],
 '«Фильмы художников», Дэвид Кертис': ['История', 'История и теория кино'],
 '«Русский консерватизм в первой четверти XIX века», Аркадий Минаков': ['История', 'История русского консерватизма'],
 '«Славянофильство: из истории русской общественно-политической мысли XIX века», Николай Цимбаев': ['История',
                                                                                                    'История русского '
                                                                                                    'консерватизма'],
 '«Русский свободный консерватизм первой половины XIX века», Владимир Шульгин': ['История',
                                                                                 'История русского консерватизма'],
 '«Русская консервативная журналистика 1870–1890-х гг.: опыт ведения общественной дискуссии», Александр Котов': ['История',
                                                                                                                 'История '
                                                                                                                 'русского '
                                                                                                                 'консерватизма'],
 '«Консервативные модели российской государственности», Александр Репников': ['История',
                                                                              'История русского консерватизма'],
 '«Нидерланды: каприз истории», Геерт Мак': ['История', 'История Нидерландов'],
 '«Голландская республика. Её подъем, величие и падение. 1477–1806: В 2 томах», Джонатан Израэль': ['История',
                                                                                                    'История '
                                                                                                    'Нидерландов'],
 '«Истории страны Рембрандта», Ольга Тилкес': ['История', 'История Нидерландов'],
 '«Королевский дом Нидерландов», Марта Шад': ['История', 'История Нидерландов'],
 '«Амстердам. История самого свободного города в мире», Рассел Шорто': ['История', 'История Нидерландов'],
 '«Orientalism in Art», Кристин Пельтр': ['История', 'Ориентализм в искусстве XIX века'],
 '«Каталог выставки Государственного Эрмитажа «Египтомания: К 200-летию дешифровки египетских иероглифов Ж.-Ф. Шампольоном», Андрей Большаков': ['История',
                                                                                                                                                 'Ориентализм '
                                                                                                                                                 'в '
                                                                                                                                                 'искусстве '
                                                                                                                                                 'XIX '
                                                                                                                                                 'века'],
 '«Ориентализм: Турецкий стиль в России 1760–1840-е», Ольга Соснина': ['История', 'Ориентализм в искусстве XIX века'],
 '«Cборник статей Института востоковедения Российской академии наук Искусство Востока и Восток в искусстве: от традиционных форм к современным арт-практикам. Art in the East and East in Arts: from traditional forms to contemporary art practices: материалы I Междунар. науч. конф.», отв. ред. Динара Дубровская': ['История',
                                                                                                                                                                                                                                                                                                                         'Ориентализм '
                                                                                                                                                                                                                                                                                                                         'в '
                                                                                                                                                                                                                                                                                                                         'искусстве '
                                                                                                                                                                                                                                                                                                                         'XIX '
                                                                                                                                                                                                                                                                                                                         'века'],
 '«Ориентализм», Эдвард Саид': ['История', 'Ориентализм в искусстве XIX века'],
 '«Повседневная жизнь опричников Ивана Грозного», Игорь Курукин, Андрей Булычев': ['История', 'Иван Грозный'],
 '«Иван Грозный, его предки и родня: История в лицах и судьбы в свете новых исследований», Татьяна Панова': ['История',
                                                                                                             'Иван '
                                                                                                             'Грозный'],
 '«Апология истории», Марк Блок': ['История', 'Методология истории'],
 '«Двенадцать уроков по истории», Антуан Про': ['История', 'Методология истории'],
 '«Что такое культуральная история?», Питер Берк': ['История', 'Методология истории'],
 '«Великорусский пахарь и особенности российского исторического процесса», Леонид Милов': ['История',
                                                                                           'История допетровской '
                                                                                           'России'],
 '«Древняя Русь IX–XIII веков. Народные движения. Княжеская и вечевая власть», Игорь Фроянов': ['История',
                                                                                                'История допетровской '
                                                                                                'России'],
 '«Дипломатия Древней Руси», Андрей Сахаров': ['История', 'История допетровской России'],
 '«Государь всея Руси», Юрий Алексеев': ['История', 'История допетровской России'],
 '«Иван Грозный» и «Борис Годунов», Руслан Скрынников': ['История', 'История допетровской России'],
 '«Машина, творящая богов», Серж Московичи': ['Психология', 'История психологии и историческая психология'],
 '«Герой с тысячью лицами: Миф. Архетип. Бессознательное», Джозеф Кэмпбелл': ['Психология',
                                                                              'История психологии и историческая '
                                                                              'психология'],
 '«Золотая ветвь: Исследование магии и религии», Джеймс Фрэзер': ['Психология',
                                                                  'История психологии и историческая психология'],
 '«Сверхъестественное в первобытном мышлении», Люсьен Леви-Брюль': ['Психология',
                                                                    'История психологии и историческая психология'],
 '«Опыт о человеке: Введение в философию человеческой культуры», Эрнст Кассирер': ['Психология',
                                                                                   'История психологии и историческая '
                                                                                   'психология'],
 '«Психологическое консультирование//4-е изд.», Сэмюэль Глэддинг': ['Психология', 'Психологическое консультирование'],
 '«Психологическое консультирование и психотерапия. Методы, теории и техники: практическое руководство», Айви Аллен, Айви Брэдфорд, Саймэк-Даунинг Линн': ['Психология',
                                                                                                                                                           'Психологическое '
                                                                                                                                                           'консультирование'],
 '«Совершенный психотерапевт. Работа с трудными клиентами», Джеффри Коттлер': ['Психология',
                                                                               'Психологическое консультирование'],
 '«Психотерапия — это не то, что вы думаете: Психотерапевтическое взаимодействие в живом мгновении», Джеймс Бьюдженталь': ['Психология',
                                                                                                                           'Психологическое '
                                                                                                                           'консультирование'],
 '«Дар психотерапии», Ирвин Ялом': ['Психология', 'Психологическое консультирование'],
 '«Стресс без дистресса», Ганс Селье': ['Психология', 'Психология стресса и психическая травма'],
 '«Почему у зебр не бывает инфаркта», Роберт Сапольски': ['Психология', 'Психология стресса и психическая травма'],
 '«Пробуждение тигра. Исцеление травмы», Питер Левин': ['Психология', 'Психология стресса и психическая травма'],
 '«Тело помнит всё», Бессел Ван дер Колк': ['Психология', 'Психология стресса и психическая травма'],
 '«Комплексное ПТСР: руководство по восстановлению от детской травмы», Пит Уокер': ['Психология',
                                                                                    'Психология стресса и психическая '
                                                                                    'травма'],
 '«Свобода от тревоги. Справься с тревогой, пока она не расправилась с тобой», Роберт Лихи': ['Клиническая психология',
                                                                                              '«В кабинете '
                                                                                              'психотерапевта и вне '
                                                                                              'его». Книги для терапии '
                                                                                              'и самопомощи'],
 '«Разум рулит настроением. Измени свои мысли, привычки, здоровье, жизнь», Кристин Падески, Деннис Гринбергер': ['Клиническая '
                                                                                                                 'психология',
                                                                                                                 '«В '
                                                                                                                 'кабинете '
                                                                                                                 'психотерапевта '
                                                                                                                 'и '
                                                                                                                 'вне '
                                                                                                                 'его». '
                                                                                                                 'Книги '
                                                                                                                 'для '
                                                                                                                 'терапии '
                                                                                                                 'и '
                                                                                                                 'самопомощи'],
 '«Когнитивная терапия, ориентированная на восстановление», Аарон Бек, Пол Грант, Эллен Инверсо, Аарон Бринен, Димитрий Периволиотис': ['Клиническая '
                                                                                                                                        'психология',
                                                                                                                                        '«В '
                                                                                                                                        'кабинете '
                                                                                                                                        'психотерапевта '
                                                                                                                                        'и '
                                                                                                                                        'вне '
                                                                                                                                        'его». '
                                                                                                                                        'Книги '
                                                                                                                                        'для '
                                                                                                                                        'терапии '
                                                                                                                                        'и '
                                                                                                                                        'самопомощи'],
 '«Почему я отвлекаюсь. Как распознать синдром дефицита внимания у взрослых и детей и что с ним делать», Эдвард Хэлловэлл, Джон Рэйти': ['Клиническая '
                                                                                                                                         'психология',
                                                                                                                                         '«В '
                                                                                                                                         'кабинете '
                                                                                                                                         'психотерапевта '
                                                                                                                                         'и '
                                                                                                                                         'вне '
                                                                                                                                         'его». '
                                                                                                                                         'Книги '
                                                                                                                                         'для '
                                                                                                                                         'терапии '
                                                                                                                                         'и '
                                                                                                                                         'самопомощи'],
 '«Победить расстройство пищевого поведения. Когнитивно-поведенческая терапия при нервной булимии и психогенном переедании, пошаговое пособие по самопомощи», Стюарт Аграс, Робин Эпл': ['Клиническая '
                                                                                                                                                                                           'психология',
                                                                                                                                                                                           '«В '
                                                                                                                                                                                           'кабинете '
                                                                                                                                                                                           'психотерапевта '
                                                                                                                                                                                           'и '
                                                                                                                                                                                           'вне '
                                                                                                                                                                                           'его». '
                                                                                                                                                                                           'Книги '
                                                                                                                                                                                           'для '
                                                                                                                                                                                           'терапии '
                                                                                                                                                                                           'и '
                                                                                                                                                                                           'самопомощи'],
 '«Самооценка. Проверенная программа когнитивных техник для улучшения вашего самоуважения», Мэтью Маккей, Патрик Фаннинг': ['Клиническая '
                                                                                                                            'психология',
                                                                                                                            'Психологическое '
                                                                                                                            'благополучие, '
                                                                                                                            'психотерапия '
                                                                                                                            '— '
                                                                                                                            'книги '
                                                                                                                            'самопомощи'],
 '«Прочь из замкнутого круга! Как оставить проблемы в прошлом и впустить в свою жизнь счастье», Джеффри Янг, Джанет Клоско': ['Клиническая '
                                                                                                                              'психология',
                                                                                                                              'Психологическое '
                                                                                                                              'благополучие, '
                                                                                                                              'психотерапия '
                                                                                                                              '— '
                                                                                                                              'книги '
                                                                                                                              'самопомощи'],
 '«Подходим друг другу. Как теория привязанности поможет создать гармоничные отношения», Амир Левин, Рэйчел Хеллер': ['Клиническая '
                                                                                                                      'психология',
                                                                                                                      'Психологическое '
                                                                                                                      'благополучие, '
                                                                                                                      'психотерапия '
                                                                                                                      '— '
                                                                                                                      'книги '
                                                                                                                      'самопомощи'],
 '«Идеальный шторм. Как пережить психологический кризис», Екатерина Сигитова': ['Клиническая психология',
                                                                                'Психологическое благополучие, '
                                                                                'психотерапия — книги самопомощи'],
 '«Мамочка и смысл жизни. Психотерапевтические истории», Ирвин Ялом': ['Клиническая психология',
                                                                       'Психологическое благополучие, психотерапия — '
                                                                       'книги самопомощи']}

model, book_vecs = build_embeddings(book_db)

seed_books = [
    '«Мамочка и смысл жизни. Психотерапевтические истории», Ирвин Ялом',
]
print("Input:", " | ".join(seed_books))
print("Recommendations: \n  - " + "\n  - ".join(recommend(book_db, book_vecs, seed_books, top_k=5)))

Input: «Мамочка и смысл жизни. Психотерапевтические истории», Ирвин Ялом
Recommendations: 
  - «Самооценка. Проверенная программа когнитивных техник для улучшения вашего самоуважения», Мэтью Маккей, Патрик Фаннинг
  - «Прочь из замкнутого круга! Как оставить проблемы в прошлом и впустить в свою жизнь счастье», Джеффри Янг, Джанет Клоско
  - «Подходим друг другу. Как теория привязанности поможет создать гармоничные отношения», Амир Левин, Рэйчел Хеллер
  - «Идеальный шторм. Как пережить психологический кризис», Екатерина Сигитова
  - «Свобода от тревоги. Справься с тревогой, пока она не расправилась с тобой», Роберт Лихи
